In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import csv
import os
import math
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import ArrayType, MapType, StringType, IntegerType, DoubleType
import json
from pyspark.sql.functions import avg, mean
import ast

In [2]:
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}"
os.environ['_JAVA_OPTIONS'] = '-Dhadoop.security.authentication=simple -Duser.name=spark'

# Была проблема с безопасностью hadoop

conf = SparkConf().setAppName("hw1").setMaster("local[*]")
sc = SparkContext(conf=conf)

Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
25/11/03 14:04:18 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 10.0.3.47 instead (on interface en0)
25/11/03 14:04:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/03 14:04:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# csv_rdd = sc.textFile("/Users/ilya/Downloads/places.csv")
# for line in csv_rdd.collect():
#     print(f"{line}")

In [4]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * \
        math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [5]:
def read_cafe_csv(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)

            for row in reader:
                if len(row) > 13:
                    try:
                        name = row[1]
                        addr = row[8]
                        lat = float(row[13])
                        lng = float(row[12])
                        data.append((name, addr, lat, lng))
                    except:
                        continue
        return sc.parallelize(data)
    except Exception as e:
        print(f"Ошибка: {e}")
        return []


restaurants_list = read_cafe_csv(
    '/Users/ilya/Downloads/places.csv')

restaurants_list.take(5)

[('Шоколадница',
  'город Москва, Бауманская улица, дом 33/2, строение 1',
  55.77302899555391,
  37.678681276899106),
 ('МУ-МУ',
  'город Москва, Бауманская улица, дом 35/1',
  55.771586681103294,
  37.678960226636995),
 ('КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА',
  'город Москва, Бригадирский переулок, дом 3-5',
  55.76545455297085,
  37.679609224948955),
 ('Дом 12',
  'город Москва, улица Казакова, дом 8А',
  55.762608795785084,
  37.661498425199866),
 ('Чито-Ра',
  'город Москва, улица Казакова, дом 10/2, строение 1',
  55.7622012817049,
  37.662845514876196)]

* Задание 1(1)

In [6]:
lat = 55.751244
lng = 37.618423
distances_rdd = restaurants_list.map(lambda row: {
    'name': row[0],
    'address': row[1],
    'distance_km': haversine_distance(row[2], row[3], lat, lng)
})

In [7]:
distances_rdd.collect()

[{'name': 'Шоколадница',
  'address': 'город Москва, Бауманская улица, дом 33/2, строение 1',
  'distance_km': 4.481040074689028},
 {'name': 'МУ-МУ',
  'address': 'город Москва, Бауманская улица, дом 35/1',
  'distance_km': 4.411446758754135},
 {'name': 'КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА',
  'address': 'город Москва, Бригадирский переулок, дом 3-5',
  'distance_km': 4.14156540296493},
 {'name': 'Дом 12',
  'address': 'город Москва, улица Казакова, дом 8А',
  'distance_km': 2.976777477746649},
 {'name': 'Чито-Ра',
  'address': 'город Москва, улица Казакова, дом 10/2, строение 1',
  'distance_km': 3.0348415748590902},
 {'name': 'Бар- буфет «Николай»',
  'address': 'город Москва, Старая Басманная улица, дом 5, строение 1',
  'distance_km': 2.828729205629978},
 {'name': 'Флорентини',
  'address': 'город Москва, Старая Басманная улица, дом 9, корпус 1',
  'distance_km': 2.888578235150347},
 {'name': 'Beer Gik',
  'address': 'город Москва, Мясницкая улица, дом 22, строение 1',
  'distance

* Задание 1(2)

In [8]:
pairs_rdd = restaurants_list.cartesian(restaurants_list)
# pairs_rdd.collect()[1]
unique_pairs_rdd = pairs_rdd.filter(lambda pair: pair[0] != pair[1])
# unique_pairs_rdd.collect()[1][1]

distances_rdd = unique_pairs_rdd.map(lambda pair: {
    'cafe1': pair[0][0],
    'cafe2': pair[1][0],
    'addr1': pair[0][1],
    'addr2': pair[1][1],
    'distance': haversine_distance(pair[0][2], pair[0][3], pair[1][2], pair[1][3])
})

distances_rdd.take(10)

[{'cafe1': 'Шоколадница',
  'cafe2': 'МУ-МУ',
  'addr1': 'город Москва, Бауманская улица, дом 33/2, строение 1',
  'addr2': 'город Москва, Бауманская улица, дом 35/1',
  'distance': 0.16132425744533824},
 {'cafe1': 'Шоколадница',
  'cafe2': 'КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА',
  'addr1': 'город Москва, Бауманская улица, дом 33/2, строение 1',
  'addr2': 'город Москва, Бригадирский переулок, дом 3-5',
  'distance': 0.844237258984877},
 {'cafe1': 'Шоколадница',
  'cafe2': 'Дом 12',
  'addr1': 'город Москва, Бауманская улица, дом 33/2, строение 1',
  'addr2': 'город Москва, улица Казакова, дом 8А',
  'distance': 1.58043754878853},
 {'cafe1': 'Шоколадница',
  'cafe2': 'Чито-Ра',
  'addr1': 'город Москва, Бауманская улица, дом 33/2, строение 1',
  'addr2': 'город Москва, улица Казакова, дом 10/2, строение 1',
  'distance': 1.5591072631893412},
 {'cafe1': 'Шоколадница',
  'cafe2': 'Бар- буфет «Николай»',
  'addr1': 'город Москва, Бауманская улица, дом 33/2, строение 1',
  'addr2': 'город 

* Задание 1(3)

In [9]:
sorted_desc_rdd = distances_rdd.sortBy(
    lambda x: x['distance'], ascending=False)

sorted_desc_rdd.take(10)

[{'cafe1': 'МНИТИ',
  'cafe2': 'Calabash Club',
  'addr1': 'город Москва, Гольяновская улица, дом 7А, строение 1',
  'addr2': 'город Москва, Лубянский проезд, дом 5, строение 1',
  'distance': 5.456078051728195},
 {'cafe1': 'МНИТИ',
  'cafe2': 'Залечь на дно',
  'addr1': 'город Москва, Гольяновская улица, дом 7А, строение 1',
  'addr2': 'город Москва, Лубянский проезд, дом 5, строение 1',
  'distance': 5.456078051728195},
 {'cafe1': 'МНИТИ',
  'cafe2': 'Политех',
  'addr1': 'город Москва, Гольяновская улица, дом 7А, строение 1',
  'addr2': 'город Москва, Лубянский проезд, дом 5, строение 1',
  'distance': 5.456078051728195},
 {'cafe1': 'МНИТИ',
  'cafe2': 'Антикафе Checkpoint',
  'addr1': 'город Москва, Гольяновская улица, дом 7А, строение 1',
  'addr2': 'город Москва, Лубянский проезд, дом 5, строение 1',
  'distance': 5.456078051728195},
 {'cafe1': 'Calabash Club',
  'cafe2': 'МНИТИ',
  'addr1': 'город Москва, Лубянский проезд, дом 5, строение 1',
  'addr2': 'город Москва, Гольяновск

In [10]:
sorted_rdd = distances_rdd.sortBy(
    lambda x: x['distance'], ascending=True)

sorted_rdd.take(10)

[{'cafe1': 'Beer Gik',
  'cafe2': 'Погребок',
  'addr1': 'город Москва, Мясницкая улица, дом 22, строение 1',
  'addr2': 'город Москва, Мясницкая улица, дом 22, строение 1',
  'distance': 0.0},
 {'cafe1': 'Погребок',
  'cafe2': 'Beer Gik',
  'addr1': 'город Москва, Мясницкая улица, дом 22, строение 1',
  'addr2': 'город Москва, Мясницкая улица, дом 22, строение 1',
  'distance': 0.0},
 {'cafe1': 'TEMPO DI PASTA',
  'cafe2': 'Хлеб насущный',
  'addr1': 'город Москва, Мясницкая улица, дом 24/7, строение 1',
  'addr2': 'город Москва, Мясницкая улица, дом 24/7, строение 1',
  'distance': 0.0},
 {'cafe1': 'Хлеб насущный',
  'cafe2': 'TEMPO DI PASTA',
  'addr1': 'город Москва, Мясницкая улица, дом 24/7, строение 1',
  'addr2': 'город Москва, Мясницкая улица, дом 24/7, строение 1',
  'distance': 0.0},
 {'cafe1': 'Глав Пив Маг',
  'cafe2': 'Beermood',
  'addr1': 'город Москва, улица Макаренко, дом 2/21, строение 1',
  'addr2': 'город Москва, улица Макаренко, дом 2/21, строение 1',
  'distance'

* Задание 2(1)

In [11]:
def parse_meta(line):
    try:
        line = line.strip()

        data = ast.literal_eval(line)
        # print(data.keys())
        asin = data.get('asin', '0')
        name = data.get('title', 'unk')

        return (asin, name)
    except Exception as e:
        return None


meta_rdd = sc.textFile('/Users/ilya/Downloads/meta_Electronics.json') \
    .map(parse_meta) \
    .filter(lambda x: x is not None)


meta_rdd.take(5)

[('0132793040',
  'Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker'),
 ('0321732944',
  'Kelby Training DVD: Adobe Photoshop CS5 Crash Course By Matt Kloskowski'),
 ('0439886341', 'Digital Organizer and Messenger'),
 ('0511189877', 'CLIKR-5 Time Warner Cable Remote Control UR5U-8780L'),
 ('0528881469',
  'Rand McNally 528881469 7-inch Intelliroute TND 700 Truck GPS')]

In [12]:
def parse_reviews(line):
    try:
        line = line.strip()

        data = ast.literal_eval(line)
        # print(data.keys())
        reviewerID = data.get('reviewerID', 0)
        asin = data.get('asin', '0')
        name = data.get('reviewerName', 'unk')
        overall = data.get('overall', 0)

        return (reviewerID, asin, name, overall)
    except Exception as e:
        return None


reviews_rdd = sc.textFile("/Users/ilya/Downloads/Electronics_5.json") \
    .map(parse_reviews) \
    .filter(lambda x: x is not None)


reviews_rdd.take(5)

[('AO94DHGC771SJ', '0528881469', 'amazdnu', 5.0),
 ('AMO214LNFCEI4', '0528881469', 'Amazon Customer', 1.0),
 ('A3N7T0DY83Y4IG', '0528881469', 'C. A. Freeman', 3.0),
 ('A1H8PY3QHMQQA0', '0528881469', 'Dave M. Shaw "mack dave"', 2.0),
 ('A24EV6RXELQZ63', '0528881469', 'Wayne Smith', 1.0)]

In [13]:
overall_rdd = reviews_rdd.map(lambda x: float(
    x[3])).filter(lambda overall: overall > 0)

print(f'Средняя оценка: {overall_rdd.mean()}')

Средняя оценка: 4.222779228836578


* Задание 2(2)

In [14]:
average_ratings = reviews_rdd.map(lambda x: (x[1], (x[3], 1))).reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])).mapValues(lambda x: x[0] / x[1])

average_ratings.take(5)

[('9985511476', 4.631578947368421),
 ('B00000J1E6', 2.142857142857143),
 ('B00000K13I', 4.533333333333333),
 ('B00001OPJV', 3.2),
 ('B00001P505', 4.434210526315789)]

In [15]:
join_items = meta_rdd.rightOuterJoin(average_ratings)

In [16]:
samples = join_items.take(5)

In [17]:
for sample in samples:
    print(f'Name: {sample[1][0]}, overall: {sample[1][1]}')

Name: Targus PA400U DEFCON 1 Ultra Notebook Computer Security System, overall: 3.375
Name: Labtec LCS-1060 Spin 60 2-Piece Computer Speakers, overall: 4.666666666666667
Name: Sony DSC-S70 Cyber-shot 3.2MP Digital Camera with 3x Optical Zoom, overall: 4.777777777777778
Name: Uniden PRO520XL 40-Channel CB Radio, overall: 4.571428571428571
Name: Cisco-Linksys EFSP42 EtherFast 10/100 2-Port Switched PrintServer, overall: 3.3333333333333335


* Задание 2(3)

In [18]:
top_data = join_items.filter(lambda x: x[1][1] <= 3) \
    .map(lambda x: (x[1][0], x[1][1])).sortBy(lambda x: x[1])

top_data.take(5)

[('StarTech HDMISPL1HH 1 feet Standard HDMI Cable - 1x HDMI (M) to 2x HDMI (F) (Discontinued by Manufacturer)',
  1.0),
 ('ATI TV Wonder 200 PCI Video Card w/PVR Capabilities', 1.0),
 ('RCA DRC8335 DVD Recorder &amp; VCR Combo With Built-In Tuner', 1.0),
 ('GE 24746 Futura HDTV Ready Antenna', 1.0),
 ('NEEWER&reg; Photographic Barn Door &amp; Honeycomb Grid &amp; Gel Set for Alienbees Alienbee Flash',
  1.0)]

* Задание 2(4)

In [19]:
# top_data.saveAsTextFile("/Users/ilya/Downloads/data_test")

* Задание 3(1)

In [20]:
ratings_rdd = sc.textFile('/Users/ilya/Downloads/ml-latest-small/ratings.csv')

first_rdd = ratings_rdd.first()

ratings_rdd_filter = ratings_rdd.filter(lambda x: x != first_rdd).map(lambda x: (
    x.split(',')[0], x.split(',')[1], float(x.split(',')[2]), x.split(',')[3]))
# ratings_rdd.take(5)
# pairs_ratings_rdd = ratings_rdd.cartesian(ratings_rdd)

# pairs_ratings_rdd.take(5)[1]

movie_avg_ratings = ratings_rdd_filter.map(lambda x: (x[1], x[2])).groupByKey().mapValues(lambda ratings: (
    sum(ratings), len(ratings))).mapValues(lambda x: x[0] / x[1])

for movie_id, avg_rating in movie_avg_ratings.take(10):
    print(f'Фильм {movie_id}: средний рейтинг = {avg_rating}')

Фильм 3: средний рейтинг = 3.2596153846153846
Фильм 6: средний рейтинг = 3.946078431372549
Фильм 70: средний рейтинг = 3.5090909090909093
Фильм 110: средний рейтинг = 4.031645569620253
Фильм 151: средний рейтинг = 3.5454545454545454
Фильм 157: средний рейтинг = 2.8636363636363638
Фильм 163: средний рейтинг = 3.5606060606060606
Фильм 216: средний рейтинг = 3.326530612244898
Фильм 231: средний рейтинг = 3.0601503759398496
Фильм 235: средний рейтинг = 3.6785714285714284


In [21]:
movie_pairs = movie_avg_ratings.cartesian(
    movie_avg_ratings).filter(lambda x: x[0][0] != x[1][0])

movie_pairs.take(5)[0][0][0]

25/11/03 14:04:53 ERROR ShuffleBlockFetcherIterator: Error occurred while fetching local blocks, null


'3'

In [22]:
def rating_to_vector(rating, max_rating=5.0):
    angle = (rating / max_rating) * (math.pi / 2)
    x = math.cos(angle)
    y = math.sin(angle)

    return [x, y]


def cosine_similarity(vec1, vec2):
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    norm1 = math.sqrt(sum(a**2 for a in vec1))
    norm2 = math.sqrt(sum(a**2 for a in vec2))

    if norm1 == 0 or norm2 == 0:
        return 0.0

    return dot_product / (norm1 * norm2)

In [23]:
similarities = movie_pairs.map(lambda x: (
    (x[0][0], x[1][0]),
    cosine_similarity(rating_to_vector(x[0][1]), rating_to_vector(x[1][1]))
))

similarities.take(15)

25/11/03 14:04:53 ERROR ShuffleBlockFetcherIterator: Error occurred while fetching local blocks, null


[(('3', '6'), 0.9768356446546346),
 (('3', '70'), 0.9969302478151483),
 (('3', '110'), 0.970730972834538),
 (('3', '151'), 0.9959707766179966),
 (('3', '157'), 0.9922722341184944),
 (('3', '163'), 0.9955326270521733),
 (('3', '216'), 0.9997790450806505),
 (('3', '231'), 0.9980372676903433),
 (('3', '235'), 0.991350726715314),
 (('3', '296'), 0.9569446200626887),
 (('3', '316'), 0.9993430716454413),
 (('3', '333'), 0.9866662854772832),
 (('3', '356'), 0.9598966952794513),
 (('3', '367'), 0.9997231550278078),
 (('3', '441'), 0.9779978116178495)]

* Задание 3(2)

In [24]:
movie_589 = similarities.filter(lambda x: x[0][0] == '589')

movie_589.take(10)

[(('589', '3'), 0.9751315577334155),
 (('589', '6'), 0.9999693947672211),
 (('589', '70'), 0.9894903801349172),
 (('589', '110'), 0.9998184022417219),
 (('589', '151'), 0.9910776689416033),
 (('589', '157'), 0.9400965436243887),
 (('589', '163'), 0.9917008769240494),
 (('589', '216'), 0.9795748019602106),
 (('589', '231'), 0.9593387487274918),
 (('589', '235'), 0.9957835119167131)]

* Задание 3(3)

In [25]:
title_movie = sc.textFile('/Users/ilya/Downloads/ml-latest-small/movies.csv')
head = title_movie.first()
title_movie_rdd = title_movie.filter(lambda x: x != head).map(
    lambda x: (x.split(',')[0], x.split(',')[1]))

title_movie_rdd.take(5)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)'),
 ('4', 'Waiting to Exhale (1995)'),
 ('5', 'Father of the Bride Part II (1995)')]

In [ ]:
movies_dict = dict(title_movie_rdd.collect())


def add_both_movie_names(similarity_pair):
    (movie1_id, movie2_id), similarity = similarity_pair

    movie1_name = movies_dict.get(movie1_id)
    movie2_name = movies_dict.get(movie2_id)

    return (movie1_name, movie2_name, similarity)


res = similarities.map(add_both_movie_names)

res.take(5)

25/11/03 14:04:55 ERROR ShuffleBlockFetcherIterator: Error occurred while fetching local blocks, null


[('Grumpier Old Men (1995)', 'Heat (1995)', 0.9768356446546346),
 ('Grumpier Old Men (1995)', 'From Dusk Till Dawn (1996)', 0.9969302478151483),
 ('Grumpier Old Men (1995)', 'Braveheart (1995)', 0.970730972834538),
 ('Grumpier Old Men (1995)', 'Rob Roy (1995)', 0.9959707766179966),
 ('Grumpier Old Men (1995)', 'Canadian Bacon (1995)', 0.9922722341184944)]

* Задание 3(4)

In [28]:
res.sortBy(lambda x: x[2]).take(10)

[('Galaxy of Terror (Quest) (1981)',
  "Rust and Bone (De rouille et d'os) (2012)",
  0.15643446504023092),
 ('Alien Contamination (1980)',
  "Rust and Bone (De rouille et d'os) (2012)",
  0.15643446504023092),
 ("I'm the One That I Want (2000)",
  "Rust and Bone (De rouille et d'os) (2012)",
  0.15643446504023092),
 ('Galaxy of Terror (Quest) (1981)', '"Christmas Carol', 0.15643446504023092),
 ('Alien Contamination (1980)', '"Christmas Carol', 0.15643446504023092),
 ("I'm the One That I Want (2000)", '"Christmas Carol', 0.15643446504023092),
 ('Galaxy of Terror (Quest) (1981)', 'Sorrow (2015)', 0.15643446504023092),
 ('Galaxy of Terror (Quest) (1981)', 'Satanic (2016)', 0.15643446504023092),
 ('Galaxy of Terror (Quest) (1981)',
  'Pokemon 4 Ever (a.k.a. Pokémon 4: The Movie) (2002)',
  0.15643446504023092),
 ('Galaxy of Terror (Quest) (1981)',
  'Pokémon Heroes (2003)',
  0.15643446504023092)]